In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2Model, pipeline, GPT2LMHeadModel, GenerationConfig

## GPT2 Generate

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2Model.from_pretrained('gpt2')
text = "The Fox is being late for work. The Cat is drinking the coffee. Who is more chill?"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output.last_hidden_state.shape)

torch.Size([1, 20, 768])


In [14]:
generator = pipeline('text-generation', model='gpt2')

Device set to use cpu


In [15]:
s = 'The Fox is being late for work. The Cat is drinking the coffee. Who is more chill, the Fox or the Cat?'
s = 'What is the capital of USA?'
s = 'Given dictionary list values comma separated. Example. Input: {"abc": 1, "def": "2"}. Answer: 1, "2". Input: {"key": "pet", "key2": "xyz", "ArithmeticError": "2+2=4"}. Answer:'
# s = 'Context: The capital of the United States of America is Washington D.C. Question: What is the capital of USA? Answer:'
# s = 'Context 2+2=4, 2*2=4. Question: How much is 2 + 2? Answer:'
generator(s, max_length=100, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Given dictionary list values comma separated. Example. Input: {"abc": 1, "def": "2"}. Answer: 1, "2". Input: {"key": "pet", "key2": "xyz", "ArithmeticError": "2+2=4"}. Answer: 4. Input: "x yz", "\\t{{name}}=y\\t" Output: {"hello": {"className":"DictionaryList"}. Answer: "{main:'},
 {'generated_text': 'Given dictionary list values comma separated. Example. Input: {"abc": 1, "def": "2"}. Answer: 1, "2". Input: {"key": "pet", "key2": "xyz", "ArithmeticError": "2+2=4"}. Answer: 1, "2". Input: {"key": "xyz", "key2": "math", "arithmeticError": "2+1=8"}. Result: 1, "'},
 {'generated_text': 'Given dictionary list values comma separated. Example. Input: {"abc": 1, "def": "2"}. Answer: 1, "2". Input: {"key": "pet", "key2": "xyz", "ArithmeticError": "2+2=4"}. Answer: 1, "4". Example. Input: {"foo": "3", "bar": 1}. Answer: 3. Example. Input: {"abc": "a", "def":'},
 {'generated_text': 'Given dictionary list values comma separated. Example. Input: {"abc": 1, "def": "2"}. Answer:

In [19]:
model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [5]:
model_lmh = GPT2LMHeadModel.from_pretrained('gpt2')
model_lmh

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
model_lmh.config

GPT2Config {
  "_attn_implementation_autoset": true,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 50257
}

In [20]:
gen_cfg = GenerationConfig(
    max_new_tokens=20,
    eos_token_id=tokenizer.sep_token_id,
    do_sample=True,
    top_p=0.95,
    top_k=50,
    # temperature=0.6,
)

In [24]:
text = "The Fox is being late for work. The Cat is drinking the coffee. Who is more chill?"
text = 'Who is the president of the United States?'
encoded_input = tokenizer(text, return_tensors='pt')
input_ids = encoded_input.input_ids

In [25]:
attention_mask = torch.ones(input_ids.shape, device=input_ids.device, dtype=bool)
ouput_ids = model_lmh.generate(input_ids=input_ids, attention_mask=attention_mask)
out = tokenizer.decode(ouput_ids.squeeze())
print(out)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Who is the president of the United States?

The president of the United States is the president of the United States.

The president


## GPT2 Train

In [11]:
import os
from pathlib import Path

from datasets import load_dataset
from regex import T
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [12]:
DATA_PATH = Path(os.path.expandvars('$HOME')) / 'data'

gpt2_train_path = DATA_PATH / 'gpt2_train'


In [13]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
wiki_ds_name, wiki_ds_subdir = '20220301.en', 'wikipedia'
# dataset = load_dataset(wiki_ds_subdir, wiki_ds_name, cache_dir=str(DATA_PATH))
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', cache_dir=str(DATA_PATH))

Using device: cuda


In [14]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [15]:
# Load tokenizer and model

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    inputs =  tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

In [41]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=str(gpt2_train_path),
    # evaluation_strategy='epoch',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=str(gpt2_train_path),
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Train the model
# trainer.train(resume_from_checkpoint=str(gpt2_train_path))
trainer.train()

# save the model and tokenizer explicitly
model_output_dir = str(gpt2_train_path)

model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(model_output_dir)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.730300
1000,1.350900
1500,1.381300
2000,1.279800
2500,1.271700
3000,1.250000
3500,1.281100
4000,1.306800
4500,1.260600
5000,1.264300


('/home/misha/data/gpt2_train/tokenizer_config.json',
 '/home/misha/data/gpt2_train/special_tokens_map.json',
 '/home/misha/data/gpt2_train/vocab.json',
 '/home/misha/data/gpt2_train/merges.txt',
 '/home/misha/data/gpt2_train/added_tokens.json',
 '/home/misha/data/gpt2_train/tokenizer.json')

In [42]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [44]:
text = "The Fox is being late for work. The Cat is drinking the coffee. Who is more chill?"
text = 'Who is the president of the United States?'
text = 'The president of the United States is '
encoded_input = tokenizer(text, return_tensors='pt')
input_ids = encoded_input.input_ids
attention_mask = torch.ones(input_ids.shape, device=input_ids.device, dtype=bool)
ouput_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
out = tokenizer.decode(ouput_ids.squeeze())
print(out)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The president of the United States is 
<|endoftext|>


In [23]:
dataset['train'][121]

{'text': ''}